<h1> Softmax Regression from Scratch on the Iris Dataset </h1>

In [1]:
import numpy as np
from sklearn import datasets

iris = datasets.load_iris()

X = iris['data']
y = iris['target']

In [2]:
#scaling the entire dataset

X_average = X.mean(axis = 0)
X_std = X.std(axis = 0)

In [3]:
X_scaled = (X - X_average)/X_std

Data has been scaled. 
Now we add in a bias term.

In [4]:
X_with_bias = np.c_[np.ones(shape = (len(X),1)),X_scaled]

In [5]:
np.random.seed(2042)

In [6]:
test_ratio = 0.2
validation_ratio = 0.2
total_size = X_with_bias.shape[0]

test_size = int(test_ratio*total_size)
validation_size = int(validation_ratio*total_size)
train_size = int(total_size*(1 - validation_ratio - test_ratio))

In [7]:
idx_no = np.random.permutation(total_size)

test_idx = idx_no[:test_size+1]
validation_idx = idx_no[test_size+1: test_size + validation_size + 1]
train_idx = idx_no[test_size + validation_size + 1:]

In [8]:
X_train = X_with_bias[train_idx]
y_train = y[train_idx]

In [9]:
X_val = X_with_bias[validation_idx]
y_val = y[validation_idx]

In [10]:
X_test = X_with_bias[test_idx]
y_test = y[test_idx]

In Softmax regression, as every class will have its own set of parameters, each class will have its own one-hot-encoded column vector where 1 means the instance is of that particular class and 0 means that it belongs to a different class. 

In [11]:
def one_hot_encoding(y_label):
    num_classes = y_label.max() + 1
    num_instances = len(y_label)
    Y_one_hot = np.zeros(shape = (num_instances, num_classes))
    Y_one_hot[np.arange(num_instances),y_label] = 1
    return Y_one_hot

In [12]:
one_hot_encoding(y_train[:10])

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

The one hot encoder is working now

$\sigma\left(\mathbf{s}(\mathbf{x})\right)_k = \dfrac{\exp\left(s_k(\mathbf{x})\right)}{\sum\limits_{j=1}^{K}{\exp\left(s_j(\mathbf{x})\right)}}$



In [13]:
def softmax(X, theta):
    score_matrix = np.dot(X,theta) #logit
    exp_matrix = np.exp(score_matrix)
    exp_sum = np.sum(exp_matrix, axis = 1, keepdims = True)
    return exp_matrix/exp_sum 

In [14]:
#initializing theta 
n_classes = y_train.max() + 1
n_parameters = X_with_bias.shape[1]
theta = np.random.randn(n_parameters,n_classes)
theta

array([[ 0.11330361, -0.23452355, -0.20774285],
       [ 0.43433246, -0.66647126, -0.71757054],
       [ 1.0188498 ,  0.41245226, -0.75018439],
       [ 0.63844637,  0.61608862,  0.53739165],
       [ 2.16968434,  0.08167019, -0.04731371]])

In [15]:
y_train_onehot = one_hot_encoding(y_train)

In [16]:
#training
num_iterations = 5001
m = len(X_train)
eta = 0.01

for iteration in range(num_iterations):
    y_probab = softmax(X_train,theta)
    loss = -(1/m)*(np.sum(np.log(y_probab)*y_train_onehot))
    error = y_probab - y_train_onehot 
    if iteration%100 == 0:
        print(iteration, loss)
    gradient = (1/m) * np.dot(np.transpose(X_train),error)
    theta = theta - eta*gradient

0 3.1147066270967874
100 1.1963805374003322
200 0.6624494272169731
300 0.5026173544385019
400 0.43286463445455253
500 0.3936400058710335
600 0.367909231023225
700 0.34925268284118627
800 0.33475123171543847
900 0.32289987267298476
1000 0.31285004934242716
1100 0.3040897516882617
1200 0.2962929261084237
1300 0.2892425059327578
1400 0.28278843442827795
1500 0.2768235326981102
1600 0.2712690050779982
1700 0.2660654044880077
1800 0.26116681911125317
1900 0.25653702706377907
2000 0.25214689032529997
2100 0.2479725500304892
2200 0.2439941522494359
2300 0.24019493235224745
2400 0.23656054633924437
2500 0.23307857516367458
2600 0.22973815211062398
2700 0.22652967895493362
2800 0.22344460700990149
2900 0.22047526618624688
3000 0.21761472997981793
3100 0.21485670763883996
3200 0.2121954571050609
3300 0.20962571399079577
3400 0.2071426330535259
3500 0.20474173950145896
3600 0.20241888810291933
3700 0.20017022854567182
3800 0.19799217584541826
3900 0.1958813848683037
4000 0.1938347282335308
4100 0

In [17]:
theta

array([[-0.4465294 ,  1.1398315 , -1.02226489],
       [-1.31935586,  0.14261787,  0.22702864],
       [ 1.5056302 , -0.44410561, -0.38040692],
       [-1.51685068,  1.1429574 ,  2.16581991],
       [-0.02289739, -0.33935642,  2.56629463]])

In [18]:
np.sum(np.log(y_probab)*y_train_onehot)

-15.693416235970922

In [19]:
X_train

array([[ 1.00000000e+00, -1.62768839e+00, -1.74335684e+00,
        -1.39706395e+00, -1.18381211e+00],
       [ 1.00000000e+00, -1.02184904e+00, -2.43394714e+00,
        -1.46640561e-01, -2.62386821e-01],
       [ 1.00000000e+00, -4.16009689e-01, -1.28296331e+00,
         1.37546573e-01,  1.32509732e-01],
       [ 1.00000000e+00,  1.15917263e+00, -1.31979479e-01,
         9.90107977e-01,  1.18556721e+00],
       [ 1.00000000e+00,  1.28034050e+00,  3.28414053e-01,
         1.10378283e+00,  1.44883158e+00],
       [ 1.00000000e+00,  6.74501145e-01, -5.92373012e-01,
         1.04694540e+00,  1.31719939e+00],
       [ 1.00000000e+00, -5.37177559e-01,  1.47939788e+00,
        -1.28338910e+00, -1.31544430e+00],
       [ 1.00000000e+00, -1.73673948e-01,  1.70959465e+00,
        -1.16971425e+00, -1.18381211e+00],
       [ 1.00000000e+00, -1.38535265e+00,  3.28414053e-01,
        -1.22655167e+00, -1.31544430e+00],
       [ 1.00000000e+00, -5.25060772e-02, -8.22569778e-01,
         7.62758269e-01

In [20]:
theta

array([[-0.4465294 ,  1.1398315 , -1.02226489],
       [-1.31935586,  0.14261787,  0.22702864],
       [ 1.5056302 , -0.44410561, -0.38040692],
       [-1.51685068,  1.1429574 ,  2.16581991],
       [-0.02289739, -0.33935642,  2.56629463]])

In [21]:
theta_res1 = theta

Making Predictions using the Softmax Regression results

In [24]:
score_val = softmax(X_val,theta_res1)
results_val = np.argmax(score_val, axis = 1)
results_val

array([0, 2, 1, 1, 2, 1, 0, 0, 1, 2, 2, 2, 1, 0, 2, 0, 2, 2, 1, 2, 0, 0,
       1, 1, 2, 0, 0, 1, 0, 2], dtype=int64)

In [25]:
y_val

array([0, 2, 1, 1, 2, 1, 0, 0, 1, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 0, 0,
       1, 1, 2, 0, 0, 1, 0, 2])

In [27]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_val,results_val)

array([[10,  0,  0],
       [ 0,  7,  0],
       [ 0,  2, 11]], dtype=int64)

In [28]:
from sklearn.metrics import accuracy_score

accuracy_score(y_val,results_val)

0.9333333333333333

Adding Regularization Parameter

In [69]:
n_classes = y_train.max() + 1
n_parameters = X_with_bias.shape[1]
theta_with_reg = np.random.randn(n_parameters,n_classes)
theta_with_reg

array([[-0.27467813,  0.55411656,  0.91351585],
       [ 1.36483412, -1.96477116, -1.10033411],
       [-1.73220624, -0.73448618,  0.47006739],
       [-0.08639488, -0.32797291,  0.95578971],
       [-0.18387189, -1.49720363,  0.65035392]])

In [70]:
#training
num_iterations = 5001
m = len(X_train)
eta = 0.1
epsilon = 1e-07
alpha = 0.1 #regularization 

for iteration in range(num_iterations):
    y_probab = softmax(X_train,theta_with_reg)
    cost_func = -(1/m)*(np.sum(np.log(y_probab)*y_train_onehot)) 
    reg_loss = (1/(2*m))*np.sum(np.square(theta_with_reg[1:]))
    loss = cost_func + (alpha * reg_loss)
    error = y_probab - y_train_onehot 
    if iteration%100 == 0:
        print(iteration, loss)
    cost_gradient = (1/m) * np.dot(np.transpose(X_train),error) 
    reg_gradient = (alpha/m) * np.vstack((np.zeros(shape = (1,n_classes)),theta_with_reg[1:]))
    gradient = cost_gradient + reg_gradient
    theta_with_reg = theta_with_reg - eta*gradient

theta_with_reg

0 3.2562446607869413
100 0.31592464350595484
200 0.2624762350691646
300 0.23015232317993664
400 0.20778162216172663
500 0.1914637460193028
600 0.17914010618145662
700 0.16957687580676903
800 0.16198764057115825
900 0.15585015754059867
1000 0.15080618361220224
1100 0.14660311703009019
1200 0.1430585129150659
1300 0.14003776986119498
1400 0.1374396909349138
1500 0.13518690015823842
1600 0.13321933480290893
1700 0.13148973387274335
1800 0.12996045094549785
1900 0.12860116353555345
2000 0.1273872006828902
2100 0.12629830417658255
2200 0.1253176987432234
2300 0.12443138558164397
2400 0.12362759952569875
2500 0.12289638757851448
2600 0.12222927851479658
2700 0.12161902154746146
2800 0.12105937789480728
2900 0.12054495324500954
3000 0.12007106211355348
3100 0.11963361727438916
3200 0.11922903905427573
3300 0.1188541804755061
3400 0.11850626512908
3500 0.11818283533887332
3600 0.117881708694779
3700 0.11760094143040917
3800 0.11733879742871198
3900 0.11709372187869632
4000 0.11686431879459075


array([[ 0.00880104,  3.563133  , -2.37897977],
       [-1.38007534,  0.32733129,  0.08341771],
       [ 1.13394694, -0.5921483 , -1.68007663],
       [-3.08432686,  0.18487337,  3.20811869],
       [-2.92997921, -1.79587191,  4.13823567]])

In [67]:
score_val = softmax(X_val,theta_with_reg)
results_val_reg = np.argmax(score_val, axis = 1)
results_val_reg

array([0, 2, 1, 1, 2, 1, 0, 0, 1, 2, 2, 2, 1, 0, 2, 0, 2, 2, 1, 2, 0, 0,
       1, 1, 2, 0, 0, 1, 0, 2], dtype=int64)

In [68]:
confusion_matrix(y_val, results_val_reg)

array([[10,  0,  0],
       [ 0,  7,  0],
       [ 0,  2, 11]], dtype=int64)

In [43]:
n_parameters

5

In [52]:
np.r_[np.zeros([1, n_classes]), alpha * theta_with_reg[1:]]

array([[ 0.        ,  0.        ,  0.        ],
       [-0.18977723,  0.07635881,  0.05053192],
       [ 0.197505  , -0.00872053, -0.11656378],
       [-0.29995985, -0.00420465,  0.30556469],
       [-0.19439606, -0.12811991,  0.45659941]])

In [54]:
alpha*np.vstack((np.zeros(shape = (1,n_classes)),theta_with_reg[1:]))

array([[ 0.        ,  0.        ,  0.        ],
       [-0.18977723,  0.07635881,  0.05053192],
       [ 0.197505  , -0.00872053, -0.11656378],
       [-0.29995985, -0.00420465,  0.30556469],
       [-0.19439606, -0.12811991,  0.45659941]])

Softmax Regression with Early Stopping

In [86]:
y_val

no_classes = np.max(y_val) + 1
no_instances = y_val.shape[0]

y_val_onehot = np.zeros(shape = (no_instances, no_classes))
y_val_onehot[np.arange(no_instances),y_val] = 1
y_val_onehot

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [80]:
y_val

array([0, 2, 1, 1, 2, 1, 0, 0, 1, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 0, 0,
       1, 1, 2, 0, 0, 1, 0, 2])

In [77]:
y_val.shape

(30,)

In [104]:
theta_early = np.random.randn(n_parameters,n_classes)

In [105]:
eta = 0.1
n_iterations = 5001
m = len(X_train)
alpha = 0.1
best_loss = np.infty



for iteration in range(n_iterations):
    y_probab = softmax(X_train,theta_early)
    cost_loss = (-1/m) * np.sum(np.log(y_probab)*y_train_onehot)
    reg_loss = (alpha/(2*m)) * np.sum(np.square(theta_early))
    loss = cost_func + reg_loss
    error = y_probab - y_train_onehot
    cost_grad = (1/m)*np.dot(np.transpose(X_train),error)
    reg_grad = (alpha/m)*np.vstack((np.zeros(shape = (1,n_classes)),theta_with_reg[1:]))
    gradient = cost_grad + reg_grad
    theta_early = theta_early - eta*gradient
    
    
    y_probab_val = softmax(X_val,theta_early)
    cost_loss_val = (-1/m) * np.sum(np.log(y_probab_val)*y_val_onehot)
    reg_loss_val = (alpha/(2*m)) * np.sum(np.square(theta_early))
    loss_val = cost_loss_val + reg_loss_val
    if iteration%500 == 0:
        print(iteration,loss)
    if loss_val < best_loss:
        best_loss = loss_val
    else:
        print(iteration - 1, best_loss)
        print(iteration, loss_val, 'Early Stop')
        break

theta_early

0 0.09137964857719465
500 0.09785357798534092
1000 0.10367444883927036
1388 0.06788885940353104
1389 0.06788886583032315 Early Stop


array([[-0.55111335,  2.54133878, -1.27174439],
       [-1.73123914,  0.73574931,  0.44353022],
       [ 1.45805029, -0.48546951, -0.87369695],
       [-3.51336959, -0.19333641,  2.33333496],
       [-1.06606508, -1.58209491,  2.17680252]])

In [106]:
score_val = softmax(X_val,theta_early)
results_val_reg = np.argmax(score_val, axis = 1)
results_val_reg

array([0, 2, 1, 1, 2, 1, 0, 0, 1, 2, 2, 2, 2, 0, 2, 0, 2, 2, 1, 2, 0, 0,
       1, 1, 2, 0, 0, 1, 0, 2], dtype=int64)

In [107]:
confusion_matrix(y_val, results_val_reg)

array([[10,  0,  0],
       [ 0,  7,  0],
       [ 0,  1, 12]], dtype=int64)